In [1]:
import gdal
import rasterio as rio
import numpy as np
import os, sys

In [2]:
def createArray(arr_shape):
    ''' createArray(arr_shape) creates a numpy ndarray with the shape defined by the tuple arr_shape
        arr_shape should be:
        (rows, cols, [channels])
        the values for each channel will be the number of that channel.
    '''
    
    dummy_arr = np.ones(arr_shape)
    
    if len(arr_shape) == 2:
        
        # there is only one channel, so that one channel is all 1's
        dummy_arr = np.expand_dims(dummy_arr,-1)
    
    else:
        # multiply each channel by the channel number (1-based)
        for i in range(arr_shape[-1]):
            dummy_arr[:,:,i] *= i+1
    
    return dummy_arr

        

In [3]:
createArray([5,5,1]).shape, createArray([4,6,3]).shape, createArray([8,9]).shape, createArray([2,3,4])

((5, 5, 1), (4, 6, 3), (8, 9, 1), array([[[1., 2., 3., 4.],
         [1., 2., 3., 4.],
         [1., 2., 3., 4.]],
 
        [[1., 2., 3., 4.],
         [1., 2., 3., 4.],
         [1., 2., 3., 4.]]]))

In [4]:
# initialize a "unit" geotransform array
a_geo_transform = np.zeros((3,3))

# set the last element to 1
a_geo_transform[2,2] = 1

# assign affine parameters
a_geo_transform[0,0] = 0 #/* top left x */
a_geo_transform[0,1] = 1 #/* w-e pixel resolution */
a_geo_transform[0,2] = 0 #/* rotation, 0 if image is "north up" */
a_geo_transform[1,0] = 0 #/* top left y */
a_geo_transform[1,1] = 0 #/* rotation, 0 if image is "north up" */
a_geo_transform[1,2] = 1 #/* n-s pixel resolution */

# check it
print(a_geo_transform)



[[0. 1. 0.]
 [0. 0. 1.]
 [0. 0. 1.]]


In [5]:
# define a dummy projection
a_projection=None

In [6]:
dummy = createArray([6,6,3])
n_rows, n_cols, n_channels = dummy.shape
n_rows, n_cols, n_channels

(6, 6, 3)

In [7]:
dst_filename = 'xxx.tiff'
x_pixels = n_rows  # number of pixels in x
y_pixels = n_cols  # number of pixels in y

print('getting driver name')
driver = gdal.GetDriverByName('GTiff')

getting driver name


In [9]:
help(driver.Create)

Help on method Create in module osgeo.gdal:

Create(*args, **kwargs) method of osgeo.gdal.Driver instance
    Create(Driver self, char const * utf8_path, int xsize, int ysize, int bands=1, GDALDataType eType, char ** options=None) -> Dataset



In [ ]:
print('creating dataset')
dataset = driver.Create(dst_filename, x_pixels, y_pixels, n_channels,gdal.GDT_Float32)

creating dataset


In [ ]:
print('writing')
for i in range(n_channels):
    dataset.GetRasterBand(i).WriteArray(dummy[:,:,i])

print('setting data')
# follow code is adding GeoTranform and Projection
geotrans=a_geo_transform  #get GeoTranform from existed 'data0'
proj=a_projection         #you can get from a exsited tif or import 
dataset.SetGeoTransform(geotrans)
dataset.SetProjection(proj)
dataset.FlushCache()
dataset=None

getting driver name
creating dataset
